# 📈 Validação de KPIs - Pipeline SPTrans

Este notebook valida os principais KPIs (Key Performance Indicators) do pipeline de dados SPTrans.

## Objetivos
1. Validar KPIs operacionais do transporte público
2. Analisar métricas de qualidade de dados
3. Verificar SLAs e performance do pipeline
4. Gerar dashboards de acompanhamento
5. Identificar oportunidades de melhoria

---

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine
import warnings
from datetime import datetime, timedelta
from pathlib import Path
import json
import os
from dotenv import load_dotenv

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 8)

# Carregar variáveis de ambiente
load_dotenv('../.env')

print("✅ Bibliotecas importadas com sucesso!")

## 1️⃣ Conexão com o Banco de Dados

In [ ]:
# Configurações do PostgreSQL
DB_HOST = os.getenv('POSTGRES_HOST', 'localhost')
DB_PORT = os.getenv('POSTGRES_PORT', '5432')
DB_NAME = os.getenv('POSTGRES_DB', 'sptrans_serving')
DB_USER = os.getenv('POSTGRES_USER', 'admin')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'admin123')

# String de conexão
connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

try:
    # Criar engine
    engine = create_engine(connection_string)
    
    # Testar conexão
    with engine.connect() as conn:
        result = conn.execute('SELECT version();')
        version = result.fetchone()[0]
        print(f"✅ Conectado ao PostgreSQL")
        print(f"   Versão: {version}")
        print(f"   Host: {DB_HOST}:{DB_PORT}")
        print(f"   Database: {DB_NAME}")
        
except Exception as e:
    print(f"❌ Erro ao conectar ao banco de dados: {e}")
    print("\nVerifique:")
    print("  1. PostgreSQL está rodando")
    print("  2. Credenciais no arquivo .env")
    print("  3. Database 'sptrans_serving' existe")
    engine = None

## 2️⃣ KPIs Operacionais - Transporte Público

In [ ]:
print("🚌 KPIs OPERACIONAIS - TRANSPORTE PÚBLICO")
print("=" * 80)

kpis_operacionais = {}

if engine:
    try:
        # 1. Total de Rotas Ativas
        query = """
        SELECT COUNT(DISTINCT route_id) as total_rotas
        FROM routes
        WHERE is_active = true;
        """
        df = pd.read_sql(query, engine)
        total_rotas = df['total_rotas'].iloc[0] if not df.empty else 0
        kpis_operacionais['total_rotas_ativas'] = total_rotas
        print(f"\n📊 Total de Rotas Ativas: {total_rotas:,}")
        
    except Exception as e:
        print(f"⚠️ Tabela 'routes' não disponível: {e}")

    try:
        # 2. Total de Paradas
        query = """
        SELECT COUNT(DISTINCT stop_id) as total_paradas
        FROM stops;
        """
        df = pd.read_sql(query, engine)
        total_paradas = df['total_paradas'].iloc[0] if not df.empty else 0
        kpis_operacionais['total_paradas'] = total_paradas
        print(f"\n🚏 Total de Paradas: {total_paradas:,}")
        
    except Exception as e:
        print(f"⚠️ Tabela 'stops' não disponível: {e}")

    try:
        # 3. Veículos Ativos (última hora)
        query = """
        SELECT COUNT(DISTINCT vehicle_id) as veiculos_ativos
        FROM vehicle_positions
        WHERE timestamp >= NOW() - INTERVAL '1 hour';
        """
        df = pd.read_sql(query, engine)
        veiculos_ativos = df['veiculos_ativos'].iloc[0] if not df.empty else 0
        kpis_operacionais['veiculos_ativos_1h'] = veiculos_ativos
        print(f"\n🚍 Veículos Ativos (última hora): {veiculos_ativos:,}")
        
    except Exception as e:
        print(f"⚠️ Tabela 'vehicle_positions' não disponível: {e}")

    try:
        # 4. Posições Capturadas (hoje)
        query = """
        SELECT COUNT(*) as total_posicoes
        FROM vehicle_positions
        WHERE DATE(timestamp) = CURRENT_DATE;
        """
        df = pd.read_sql(query, engine)
        posicoes_hoje = df['total_posicoes'].iloc[0] if not df.empty else 0
        kpis_operacionais['posicoes_capturadas_hoje'] = posicoes_hoje
        print(f"\n📍 Posições Capturadas (hoje): {posicoes_hoje:,}")
        
    except Exception as e:
        print(f"⚠️ Erro ao consultar posições de hoje: {e}")

else:
    print("⚠️ Banco de dados não conectado. Usando dados simulados...")
    
    # Dados simulados para demonstração
    kpis_operacionais = {
        'total_rotas_ativas': 1350,
        'total_paradas': 22000,
        'veiculos_ativos_1h': 8500,
        'posicoes_capturadas_hoje': 2450000
    }
    
    print(f"\n📊 Total de Rotas Ativas: {kpis_operacionais['total_rotas_ativas']:,}")
    print(f"\n🚏 Total de Paradas: {kpis_operacionais['total_paradas']:,}")
    print(f"\n🚍 Veículos Ativos (última hora): {kpis_operacionais['veiculos_ativos_1h']:,}")
    print(f"\n📍 Posições Capturadas (hoje): {kpis_operacionais['posicoes_capturadas_hoje']:,}")

print("\n" + "=" * 80)

In [ ]:
# Visualização dos KPIs Operacionais
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# KPI 1: Rotas Ativas
axes[0, 0].bar(['Rotas Ativas'], [kpis_operacionais['total_rotas_ativas']], color='steelblue', width=0.5)
axes[0, 0].set_title('Total de Rotas Ativas', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Quantidade')
axes[0, 0].text(0, kpis_operacionais['total_rotas_ativas']/2, 
                f"{kpis_operacionais['total_rotas_ativas']:,}", 
                ha='center', va='center', fontsize=20, fontweight='bold', color='white')

# KPI 2: Paradas
axes[0, 1].bar(['Paradas'], [kpis_operacionais['total_paradas']], color='coral', width=0.5)
axes[0, 1].set_title('Total de Paradas', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Quantidade')
axes[0, 1].text(0, kpis_operacionais['total_paradas']/2, 
                f"{kpis_operacionais['total_paradas']:,}", 
                ha='center', va='center', fontsize=20, fontweight='bold', color='white')

# KPI 3: Veículos Ativos
axes[1, 0].bar(['Veículos (1h)'], [kpis_operacionais['veiculos_ativos_1h']], color='seagreen', width=0.5)
axes[1, 0].set_title('Veículos Ativos (Última Hora)', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Quantidade')
axes[1, 0].text(0, kpis_operacionais['veiculos_ativos_1h']/2, 
                f"{kpis_operacionais['veiculos_ativos_1h']:,}", 
                ha='center', va='center', fontsize=20, fontweight='bold', color='white')

# KPI 4: Posições Capturadas
axes[1, 1].bar(['Posições (hoje)'], [kpis_operacionais['posicoes_capturadas_hoje']], color='purple', width=0.5)
axes[1, 1].set_title('Posições Capturadas (Hoje)', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Quantidade')
axes[1, 1].text(0, kpis_operacionais['posicoes_capturadas_hoje']/2, 
                f"{kpis_operacionais['posicoes_capturadas_hoje']:,}", 
                ha='center', va='center', fontsize=20, fontweight='bold', color='white')

plt.tight_layout()
plt.show()

## 3️⃣ KPIs de Qualidade de Dados

In [ ]:
print("✅ KPIs DE QUALIDADE DE DADOS")
print("=" * 80)

kpis_qualidade = {}

if engine:
    try:
        # 1. Completude - Posições com coordenadas válidas
        query = """
        SELECT 
            COUNT(*) as total,
            COUNT(latitude) as com_lat,
            COUNT(longitude) as com_lon,
            ROUND(100.0 * COUNT(latitude) / COUNT(*), 2) as completude_lat,
            ROUND(100.0 * COUNT(longitude) / COUNT(*), 2) as completude_lon
        FROM vehicle_positions
        WHERE DATE(timestamp) = CURRENT_DATE;
        """
        df = pd.read_sql(query, engine)
        if not df.empty:
            completude = (df['completude_lat'].iloc[0] + df['completude_lon'].iloc[0]) / 2
            kpis_qualidade['completude_coordenadas'] = completude
            print(f"\n📊 Completude de Coordenadas: {completude:.2f}%")
        
    except Exception as e:
        print(f"⚠️ Erro ao calcular completude: {e}")

    try:
        # 2. Acurácia - Coordenadas dentro do perímetro de SP
        query = """
        SELECT 
            COUNT(*) as total,
            COUNT(*) FILTER (
                WHERE latitude BETWEEN -24.0 AND -23.3
                AND longitude BETWEEN -46.9 AND -46.3
            ) as dentro_sp,
            ROUND(100.0 * COUNT(*) FILTER (
                WHERE latitude BETWEEN -24.0 AND -23.3
                AND longitude BETWEEN -46.9 AND -46.3
            ) / COUNT(*), 2) as acuracia
        FROM vehicle_positions
        WHERE DATE(timestamp) = CURRENT_DATE
        AND latitude IS NOT NULL
        AND longitude IS NOT NULL;
        """
        df = pd.read_sql(query, engine)
        if not df.empty:
            acuracia = df['acuracia'].iloc[0]
            kpis_qualidade['acuracia_localizacao'] = acuracia
            print(f"\n🎯 Acurácia de Localização: {acuracia:.2f}%")
        
    except Exception as e:
        print(f"⚠️ Erro ao calcular acurácia: {e}")

    try:
        # 3. Consistência - Registros duplicados
        query = """
        WITH duplicates AS (
            SELECT vehicle_id, timestamp, COUNT(*) as cnt
            FROM vehicle_positions
            WHERE DATE(timestamp) = CURRENT_DATE
            GROUP BY vehicle_id, timestamp
            HAVING COUNT(*) > 1
        )
        SELECT 
            (SELECT COUNT(*) FROM vehicle_positions WHERE DATE(timestamp) = CURRENT_DATE) as total,
            COALESCE(SUM(cnt), 0) as duplicados,
            ROUND(100.0 * (1 - COALESCE(SUM(cnt), 0)::float / 
                (SELECT COUNT(*) FROM vehicle_positions WHERE DATE(timestamp) = CURRENT_DATE)), 2) as consistencia
        FROM duplicates;
        """
        df = pd.read_sql(query, engine)
        if not df.empty:
            consistencia = df['consistencia'].iloc[0]
            kpis_qualidade['consistencia_dados'] = consistencia
            print(f"\n🔗 Consistência de Dados: {consistencia:.2f}%")
        
    except Exception as e:
        print(f"⚠️ Erro ao calcular consistência: {e}")

    try:
        # 4. Atualidade - Dados recentes (última hora)
        query = """
        SELECT 
            COUNT(*) as total_hoje,
            COUNT(*) FILTER (WHERE timestamp >= NOW() - INTERVAL '1 hour') as ultima_hora,
            ROUND(100.0 * COUNT(*) FILTER (WHERE timestamp >= NOW() - INTERVAL '1 hour') / 
                NULLIF(COUNT(*), 0), 2) as atualidade
        FROM vehicle_positions
        WHERE DATE(timestamp) = CURRENT_DATE;
        """
        df = pd.read_sql(query, engine)
        if not df.empty and df['total_hoje'].iloc[0] > 0:
            atualidade = df['atualidade'].iloc[0]
            kpis_qualidade['atualidade_dados'] = atualidade
            print(f"\n⏰ Atualidade dos Dados: {atualidade:.2f}%")
        
    except Exception as e:
        print(f"⚠️ Erro ao calcular atualidade: {e}")

else:
    print("⚠️ Banco de dados não conectado. Usando dados simulados...")
    
    kpis_qualidade = {
        'completude_coordenadas': 98.75,
        'acuracia_localizacao': 99.2,
        'consistencia_dados': 99.8,
        'atualidade_dados': 95.5
    }
    
    print(f"\n📊 Completude de Coordenadas: {kpis_qualidade['completude_coordenadas']:.2f}%")
    print(f"\n🎯 Acurácia de Localização: {kpis_qualidade['acuracia_localizacao']:.2f}%")
    print(f"\n🔗 Consistência de Dados: {kpis_qualidade['consistencia_dados']:.2f}%")
    print(f"\n⏰ Atualidade dos Dados: {kpis_qualidade['atualidade_dados']:.2f}%")

# Score geral de qualidade
if kpis_qualidade:
    score_qualidade = np.mean(list(kpis_qualidade.values()))
    print(f"\n⭐ SCORE GERAL DE QUALIDADE: {score_qualidade:.2f}%")
    
    if score_qualidade >= 95:
        print("   ✅ Excelente!")
    elif score_qualidade >= 90:
        print("   ⚠️ Bom, mas pode melhorar")
    else:
        print("   ❌ Atenção necessária")

print("\n" + "=" * 80)

In [ ]:
# Visualização dos KPIs de Qualidade
if kpis_qualidade:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Gráfico de barras
    dimensoes = ['Completude', 'Acurácia', 'Consistência', 'Atualidade']
    valores = list(kpis_qualidade.values())
    cores = ['steelblue', 'coral', 'seagreen', 'purple']
    
    bars = axes[0].bar(dimensoes, valores, color=cores, alpha=0.8)
    axes[0].set_title('Dimensões de Qualidade de Dados', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Score (%)')
    axes[0].set_ylim(0, 100)
    axes[0].axhline(y=95, color='green', linestyle='--', alpha=0.5, label='Meta: 95%')
    axes[0].axhline(y=90, color='orange', linestyle='--', alpha=0.5, label='Alerta: 90%')
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    # Adicionar valores nas barras
    for bar, val in zip(bars, valores):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{val:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Gráfico de radar
    angles = np.linspace(0, 2 * np.pi, len(dimensoes), endpoint=False).tolist()
    valores_radar = valores + [valores[0]]  # Fechar o polígono
    angles += angles[:1]
    
    ax = plt.subplot(122, projection='polar')
    ax.plot(angles, valores_radar, 'o-', linewidth=2, color='steelblue', label='Atual')
    ax.fill(angles, valores_radar, alpha=0.25, color='steelblue')
    ax.plot(angles, [95]*len(angles), '--', linewidth=1, color='green', alpha=0.5, label='Meta: 95%')
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(dimensoes)
    ax.set_ylim(0, 100)
    ax.set_title('Radar de Qualidade de Dados', fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

## 4️⃣ KPIs de Performance do Pipeline

In [ ]:
print("⚡ KPIs DE PERFORMANCE DO PIPELINE")
print("=" * 80)

kpis_performance = {}

if engine:
    try:
        # 1. Latência de Ingestão (tempo médio de processamento)
        query = """
        SELECT 
            AVG(processing_time_seconds) as latencia_media,
            MIN(processing_time_seconds) as latencia_min,
            MAX(processing_time_seconds) as latencia_max,
            PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY processing_time_seconds) as p95_latencia
        FROM pipeline_metrics
        WHERE DATE(execution_date) >= CURRENT_DATE - INTERVAL '7 days'
        AND stage = 'ingestion';
        """
        df = pd.read_sql(query, engine)
        if not df.empty and df['latencia_media'].iloc[0]:
            kpis_performance['latencia_media_s'] = df['latencia_media'].iloc[0]
            kpis_performance['p95_latencia_s'] = df['p95_latencia'].iloc[0]
            print(f"\n⏱️ Latência Média de Ingestão: {df['latencia_media'].iloc[0]:.2f}s")
            print(f"   P95 Latência: {df['p95_latencia'].iloc[0]:.2f}s")
        
    except Exception as e:
        print(f"⚠️ Tabela 'pipeline_metrics' não disponível: {e}")

    try:
        # 2. Taxa de Sucesso dos Jobs
        query = """
        SELECT 
            COUNT(*) as total_execucoes,
            COUNT(*) FILTER (WHERE status = 'success') as sucesso,
            COUNT(*) FILTER (WHERE status = 'failed') as falhas,
            ROUND(100.0 * COUNT(*) FILTER (WHERE status = 'success') / COUNT(*), 2) as taxa_sucesso
        FROM pipeline_metrics
        WHERE DATE(execution_date) >= CURRENT_DATE - INTERVAL '7 days';
        """
        df = pd.read_sql(query, engine)
        if not df.empty:
            taxa_sucesso = df['taxa_sucesso'].iloc[0]
            kpis_performance['taxa_sucesso'] = taxa_sucesso
            print(f"\n✅ Taxa de Sucesso (7 dias): {taxa_sucesso:.2f}%")
            print(f"   Total de execuções: {df['total_execucoes'].iloc[0]:,}")
            print(f"   Sucessos: {df['sucesso'].iloc[0]:,}")
            print(f"   Falhas: {df['falhas'].iloc[0]:,}")
        
    except Exception as e:
        print(f"⚠️ Erro ao calcular taxa de sucesso: {e}")

    try:
        # 3. Volume de Dados Processados
        query = """
        SELECT 
            SUM(records_processed) as total_registros,
            AVG(records_processed) as media_registros,
            SUM(bytes_processed) / (1024*1024*1024) as total_gb
        FROM pipeline_metrics
        WHERE DATE(execution_date) = CURRENT_DATE;
        """
        df = pd.read_sql(query, engine)
        if not df.empty and df['total_registros'].iloc[0]:
            kpis_performance['registros_processados_hoje'] = df['total_registros'].iloc[0]
            kpis_performance['volume_gb_hoje'] = df['total_gb'].iloc[0]
            print(f"\n📊 Registros Processados (hoje): {df['total_registros'].iloc[0]:,.0f}")
            print(f"   Volume de dados: {df['total_gb'].iloc[0]:.2f} GB")
        
    except Exception as e:
        print(f"⚠️ Erro ao calcular volume de dados: {e}")

    try:
        # 4. SLA - Disponibilidade
        query = """
        SELECT 
            COUNT(*) FILTER (WHERE status = 'success' AND 
                processing_time_seconds <= 120) as dentro_sla,
            COUNT(*) as total,
            ROUND(100.0 * COUNT(*) FILTER (WHERE status = 'success' AND 
                processing_time_seconds <= 120) / COUNT(*), 2) as sla_compliance
        FROM pipeline_metrics
        WHERE DATE(execution_date) >= CURRENT_DATE - INTERVAL '7 days'
        AND stage = 'ingestion';
        """
        df = pd.read_sql(query, engine)
        if not df.empty and df['total'].iloc[0] > 0:
            sla = df['sla_compliance'].iloc[0]
            kpis_performance['sla_compliance'] = sla
            print(f"\n🎯 SLA Compliance (< 120s): {sla:.2f}%")
        
    except Exception as e:
        print(f"⚠️ Erro ao calcular SLA: {e}")

else:
    print("⚠️ Banco de dados não conectado. Usando dados simulados...")
    
    kpis_performance = {
        'latencia_media_s': 45.3,
        'p95_latencia_s': 89.7,
        'taxa_sucesso': 99.2,
        'registros_processados_hoje': 2450000,
        'volume_gb_hoje': 8.5,
        'sla_compliance': 98.8
    }
    
    print(f"\n⏱️ Latência Média de Ingestão: {kpis_performance['latencia_media_s']:.2f}s")
    print(f"   P95 Latência: {kpis_performance['p95_latencia_s']:.2f}s")
    print(f"\n✅ Taxa de Sucesso (7 dias): {kpis_performance['taxa_sucesso']:.2f}%")
    print(f"\n📊 Registros Processados (hoje): {kpis_performance['registros_processados_hoje']:,.0f}")
    print(f"   Volume de dados: {kpis_performance['volume_gb_hoje']:.2f} GB")
    print(f"\n🎯 SLA Compliance (< 120s): {kpis_performance['sla_compliance']:.2f}%")

print("\n" + "=" * 80)

In [ ]:
# Visualização dos KPIs de Performance
if kpis_performance:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Latência
    if 'latencia_media_s' in kpis_performance:
        latencias = ['Média', 'P95', 'SLA (120s)']
        valores_lat = [
            kpis_performance['latencia_media_s'],
            kpis_performance.get('p95_latencia_s', 0),
            120
        ]
        cores_lat = ['steelblue', 'coral', 'green']
        
        axes[0, 0].barh(latencias, valores_lat, color=cores_lat, alpha=0.8)
        axes[0, 0].set_title('Latências de Processamento', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Tempo (segundos)')
        axes[0, 0].grid(axis='x', alpha=0.3)
        
        for i, (lat, val) in enumerate(zip(latencias, valores_lat)):
            axes[0, 0].text(val, i, f' {val:.1f}s', va='center', fontsize=10, fontweight='bold')
    
    # 2. Taxa de Sucesso
    if 'taxa_sucesso' in kpis_performance:
        taxa = kpis_performance['taxa_sucesso']
        falha = 100 - taxa
        
        axes[0, 1].pie([taxa, falha], labels=['Sucesso', 'Falha'], 
                      autopct='%1.2f%%', startangle=90,
                      colors=['lightgreen', 'lightcoral'])
        axes[0, 1].set_title('Taxa de Sucesso dos Jobs (7 dias)', fontsize=14, fontweight='bold')
    
    # 3. Volume de Dados
    if 'volume_gb_hoje' in kpis_performance:
        axes[1, 0].bar(['Volume Processado'], [kpis_performance['volume_gb_hoje']], 
                      color='purple', width=0.5, alpha=0.8)
        axes[1, 0].set_title('Volume de Dados Processados (Hoje)', fontsize=14, fontweight='bold')
        axes[1, 0].set_ylabel('GB')
        axes[1, 0].text(0, kpis_performance['volume_gb_hoje']/2, 
                       f"{kpis_performance['volume_gb_hoje']:.2f} GB",
                       ha='center', va='center', fontsize=16, fontweight='bold', color='white')
    
    # 4. SLA Compliance
    if 'sla_compliance' in kpis_performance:
        sla = kpis_performance['sla_compliance']
        
        axes[1, 1].bar(['SLA Compliance'], [sla], color='seagreen', width=0.5, alpha=0.8)
        axes[1, 1].axhline(y=99, color='gold', linestyle='--', linewidth=2, label='Meta: 99%')
        axes[1, 1].axhline(y=95, color='orange', linestyle='--', linewidth=2, label='Mínimo: 95%')
        axes[1, 1].set_title('SLA Compliance', fontsize=14, fontweight='bold')
        axes[1, 1].set_ylabel('Percentual (%)')
        axes[1, 1].set_ylim(0, 100)
        axes[1, 1].legend()
        axes[1, 1].text(0, sla/2, f"{sla:.2f}%",
                       ha='center', va='center', fontsize=16, fontweight='bold', color='white')
    
    plt.tight_layout()
    plt.show()

## 5️⃣ Dashboard Executivo - Resumo de KPIs

In [ ]:
# Criar dashboard executivo
print("📊 DASHBOARD EXECUTIVO - RESUMO DE KPIs")
print("=" * 80)

# Preparar dados
dashboard_data = {
    'Categoria': [],
    'KPI': [],
    'Valor': [],
    'Status': []
}

# KPIs Operacionais
for key, value in kpis_operacionais.items():
    dashboard_data['Categoria'].append('Operacional')
    dashboard_data['KPI'].append(key.replace('_', ' ').title())
    dashboard_data['Valor'].append(f"{value:,}")
    dashboard_data['Status'].append('✅')

# KPIs de Qualidade
for key, value in kpis_qualidade.items():
    dashboard_data['Categoria'].append('Qualidade')
    dashboard_data['KPI'].append(key.replace('_', ' ').title())
    dashboard_data['Valor'].append(f"{value:.2f}%")
    status = '✅' if value >= 95 else ('⚠️' if value >= 90 else '❌')
    dashboard_data['Status'].append(status)

# KPIs de Performance
for key, value in kpis_performance.items():
    dashboard_data['Categoria'].append('Performance')
    kpi_name = key.replace('_', ' ').title()
    
    if 's' in key.lower():
        valor_formatado = f"{value:.2f}s"
    elif 'gb' in key.lower():
        valor_formatado = f"{value:.2f} GB"
    elif '%' in str(value) or 'taxa' in key or 'sla' in key or 'compliance' in key:
        valor_formatado = f"{value:.2f}%"
    else:
        valor_formatado = f"{value:,.0f}"
    
    dashboard_data['Valor'].append(valor_formatado)
    
    if 'taxa' in key or 'sla' in key or 'compliance' in key:
        status = '✅' if value >= 95 else ('⚠️' if value >= 90 else '❌')
    else:
        status = '✅'
    
    dashboard_data['Status'].append(status)

# Criar DataFrame
df_dashboard = pd.DataFrame(dashboard_data)

# Exibir por categoria
for categoria in df_dashboard['Categoria'].unique():
    print(f"\n{'='*80}")
    print(f"📌 {categoria.upper()}")
    print(f"{'='*80}")
    df_cat = df_dashboard[df_dashboard['Categoria'] == categoria][['KPI', 'Valor', 'Status']]
    display(df_cat.to_string(index=False))

print(f"\n{'='*80}")
print(f"\n⏰ Última atualização: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n✅ Dashboard atualizado com sucesso!")

## 6️⃣ Análise de Tendências (Últimos 7 dias)

In [ ]:
print("📈 ANÁLISE DE TENDÊNCIAS - ÚLTIMOS 7 DIAS")
print("=" * 80)

if engine:
    try:
        # Volume de dados por dia
        query = """
        SELECT 
            DATE(timestamp) as data,
            COUNT(*) as total_registros
        FROM vehicle_positions
        WHERE DATE(timestamp) >= CURRENT_DATE - INTERVAL '7 days'
        GROUP BY DATE(timestamp)
        ORDER BY data;
        """
        df_tendencia = pd.read_sql(query, engine)
        
        if not df_tendencia.empty:
            plt.figure(figsize=(14, 6))
            plt.plot(df_tendencia['data'], df_tendencia['total_registros'], 
                    marker='o', linewidth=2, markersize=8, color='steelblue')
            plt.title('Volume de Registros por Dia (Últimos 7 dias)', fontsize=14, fontweight='bold')
            plt.xlabel('Data')
            plt.ylabel('Total de Registros')
            plt.xticks(rotation=45)
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            print(f"\n📊 Média diária: {df_tendencia['total_registros'].mean():,.0f} registros")
            print(f"📈 Dia com mais dados: {df_tendencia.loc[df_tendencia['total_registros'].idxmax(), 'data']}")
            print(f"📉 Dia com menos dados: {df_tendencia.loc[df_tendencia['total_registros'].idxmin(), 'data']}")
        
    except Exception as e:
        print(f"⚠️ Erro ao analisar tendências: {e}")
else:
    print("⚠️ Banco de dados não conectado. Pulando análise de tendências...")

print("\n" + "=" * 80)

## 7️⃣ Recomendações e Ações

In [ ]:
print("💡 RECOMENDAÇÕES E AÇÕES")
print("=" * 80)

recomendacoes = []

# Análise de qualidade
if kpis_qualidade:
    score_qualidade = np.mean(list(kpis_qualidade.values()))
    if score_qualidade < 95:
        recomendacoes.append({
            'prioridade': 'ALTA',
            'categoria': 'Qualidade',
            'recomendacao': f'Score de qualidade em {score_qualidade:.2f}% - implementar validações adicionais',
            'acao': 'Revisar regras de validação e implementar alertas'
        })

# Análise de performance
if kpis_performance:
    if kpis_performance.get('taxa_sucesso', 100) < 99:
        recomendacoes.append({
            'prioridade': 'MÉDIA',
            'categoria': 'Performance',
            'recomendacao': 'Taxa de sucesso abaixo de 99% - investigar falhas',
            'acao': 'Analisar logs de erros e implementar retry logic'
        })
    
    if kpis_performance.get('latencia_media_s', 0) > 60:
        recomendacoes.append({
            'prioridade': 'MÉDIA',
            'categoria': 'Performance',
            'recomendacao': f"Latência média em {kpis_performance['latencia_media_s']:.2f}s - otimizar processamento",
            'acao': 'Revisar queries e considerar paralelização'
        })

# Recomendações gerais
recomendacoes.extend([
    {
        'prioridade': 'BAIXA',
        'categoria': 'Monitoramento',
        'recomendacao': 'Implementar dashboards em tempo real',
        'acao': 'Configurar Grafana com métricas do Prometheus'
    },
    {
        'prioridade': 'BAIXA',
        'categoria': 'Documentação',
        'recomendacao': 'Manter documentação de KPIs atualizada',
        'acao': 'Revisar e atualizar docs/07_metadata_catalog.md'
    }
])

# Exibir recomendações
df_recomendacoes = pd.DataFrame(recomendacoes)

for prioridade in ['ALTA', 'MÉDIA', 'BAIXA']:
    df_prior = df_recomendacoes[df_recomendacoes['prioridade'] == prioridade]
    if not df_prior.empty:
        print(f"\n{'='*80}")
        print(f"🚨 PRIORIDADE {prioridade}")
        print(f"{'='*80}")
        for _, row in df_prior.iterrows():
            print(f"\n📌 Categoria: {row['categoria']}")
            print(f"   Recomendação: {row['recomendacao']}")
            print(f"   Ação: {row['acao']}")

print(f"\n{'='*80}")

## 📝 Exportar Relatório de KPIs

In [ ]:
# Preparar relatório completo
report = {
    'timestamp': datetime.now().isoformat(),
    'kpis_operacionais': kpis_operacionais,
    'kpis_qualidade': kpis_qualidade,
    'kpis_performance': kpis_performance,
    'score_qualidade': float(np.mean(list(kpis_qualidade.values()))) if kpis_qualidade else None,
    'recomendacoes': recomendacoes,
    'dashboard': dashboard_data
}

# Salvar relatório JSON
report_path = '../data/samples/kpi_validation_report.json'
Path(report_path).parent.mkdir(parents=True, exist_ok=True)

with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Relatório JSON salvo em: {report_path}")

# Salvar dashboard em CSV
csv_path = '../data/samples/kpi_dashboard.csv'
df_dashboard.to_csv(csv_path, index=False)
print(f"✅ Dashboard CSV salvo em: {csv_path}")

# Salvar recomendações em CSV
rec_path = '../data/samples/kpi_recomendacoes.csv'
if recomendacoes:
    df_recomendacoes.to_csv(rec_path, index=False)
    print(f"✅ Recomendações salvas em: {rec_path}")

print(f"\n✅ Todos os relatórios exportados com sucesso!")

## 🎯 Conclusão

In [ ]:
print("🎯 CONCLUSÃO - VALIDAÇÃO DE KPIs")
print("=" * 80)

print("\n✅ Validação de KPIs concluída com sucesso!")
print("\n📊 Resumo:")
print(f"   - {len(kpis_operacionais)} KPIs operacionais validados")
print(f"   - {len(kpis_qualidade)} dimensões de qualidade avaliadas")
print(f"   - {len(kpis_performance)} métricas de performance analisadas")
print(f"   - {len(recomendacoes)} recomendações geradas")

if kpis_qualidade:
    score = np.mean(list(kpis_qualidade.values()))
    print(f"\n⭐ Score Geral de Qualidade: {score:.2f}%")

print("\n📌 Próximos passos:")
print("   1. Implementar alertas automáticos para KPIs críticos")
print("   2. Configurar dashboards em tempo real no Grafana")
print("   3. Automatizar geração de relatórios semanais")
print("   4. Revisar e ajustar metas de SLA")

print("\n" + "=" * 80)
print(f"📅 Relatório gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)